https://huggingface.co/agents-course/notebooks/blob/main/unit2/llama-index/tools.ipynb

In [1]:
# !pip install llama-index datasets llama-index-callbacks-arize-phoenix llama-index-vector-stores-chroma llama-index-llms-mlx llama-index-llms-huggingface-api -U -q

In [2]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.embeddings.huggingface import HuggingFaceInferenceAPIEmbeddings
from llama_index.llms.mlx import MLXLLM
from llama_index.tools.google import GmailToolSpec
from llama_index.vector_stores.chroma import ChromaVectorStore

# Creating a FunctionTool

In [3]:
def get_weather(location: str) -> str:
    """Useful for getting the weather for a given location."""
    print(f"Getting weather for {location}")

    return f"The weather in {location} is sunny"

In [4]:
tool = FunctionTool.from_defaults(
    get_weather,
    name="my_weather_tool",
    description="Useful for getting the weather for a given location.",
)

In [5]:
tool.call("Sofia")

Getting weather for Sofia


ToolOutput(content='The weather in Sofia is sunny', tool_name='my_weather_tool', raw_input={'args': ('Sofia',), 'kwargs': {}}, raw_output='The weather in Sofia is sunny', is_error=False)

# Creating a QueryEngineTool

In [6]:
db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
embed_model = HuggingFaceInferenceAPIEmbeddings(model_name="BAAI/bge-small-en-v1.5")
llm = MLXLLM(model_name="mlx-community/gemma-3-4b-it-8bit")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)
query_engine = index.as_query_engine(llm=llm)
tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="some useful name",
    description="some useful description",
)
await tool.acall(
    "Responds about research on the impact of AI on the future of work and society?"
)

/var/folders/r0/6h61qg4157lcdfpct0nslmd40000gn/T/ipykernel_7625/893805942.py:4: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPIEmbedding. (Deprecated in favor of `HuggingFaceInferenceAPIEmbedding` from `llama-index-embeddings-huggingface-api` which should be used instead.)
  embed_model = HuggingFaceInferenceAPIEmbeddings(model_name="BAAI/bge-small-en-v1.5")


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

ToolOutput(content='\nI am an ecologist specializing in climate change impacts on regional ecosystems, particularly those with diverse plant species. I cannot provide information on the impact of AI on the future of work and society.\n', tool_name='some useful name', raw_input={'input': 'Responds about research on the impact of AI on the future of work and society?'}, raw_output=Response(response='\nI am an ecologist specializing in climate change impacts on regional ecosystems, particularly those with diverse plant species. I cannot provide information on the impact of AI on the future of work and society.\n', source_nodes=[NodeWithScore(node=TextNode(id_='09187725-3921-436b-bcd9-46852af130ce', embedding=None, metadata={'file_path': '/Users/ljoana/repos/LLMs-Journey/Agents/data/persona_1002.txt', 'file_name': 'persona_1002.txt', 'file_type': 'text/plain', 'file_size': 122, 'creation_date': '2025-03-23', 'last_modified_date': '2025-03-23'}, excluded_embed_metadata_keys=['file_name', 'f

In [7]:
tool_spec = GmailToolSpec()
tool_spec_list = tool_spec.to_tool_list()
tool_spec_list

In [8]:
[(tool.metadata.name, tool.metadata.description) for tool in tool_spec_list]

[('load_data',
  "load_data() -> List[llama_index.core.schema.Document]\nLoad emails from the user's account."),
 ('search_messages',
  "search_messages(query: str, max_results: Optional[int] = None)\nSearches email messages given a query string and the maximum number\n        of results requested by the user\n           Returns: List of relevant message objects up to the maximum number of results.\n\n        Args:\n            query[str]: The user's query\n            max_results (Optional[int]): The maximum number of search results\n            to return.\n        "),
 ('create_draft',
  "create_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None) -> str\nCreate and insert a draft email.\n           Print the returned draft's message and id.\n           Returns: Draft object, including draft id and message meta data.\n\n        Args:\n            to (Optional[str]): The email addresses to send the message to\n            subject (Optiona